## Installing Required Packages

In [9]:
!pip install -q ultralytics

In [10]:
# !pip install -U ipywidgets #For widgets in Kaggle

In [11]:
import shutil
import os
import yaml
from ultralytics import YOLO
import os
import json
from sklearn.model_selection import train_test_split

### Moving files in working directory

In [12]:
# Define your source directories
source_images_dir = '/kaggle/input/customdataset/ObjectDetection/images/train'
yolo_annotations_dir = '/kaggle/input/customdataset/ObjectDetection/labels/train'
coco_annotations_path = '/kaggle/input/coco-annotations/instances_default.json'

# Define the output base directories
base_dir = '/kaggle/working'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')
coco_dir = os.path.join(base_dir, 'coco_annotations')

# Make sure the base directories exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)
os.makedirs(coco_dir, exist_ok=True)

# Prepare subdirectories for train and val in each base directory
for sub_dir in ['train', 'val']:
    os.makedirs(os.path.join(images_dir, sub_dir), exist_ok=True)
    os.makedirs(os.path.join(labels_dir, sub_dir), exist_ok=True)

# List and split image files
image_files = [f for f in os.listdir(source_images_dir) if f.endswith('.jpg')]
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=2)

# Load COCO annotations
with open(coco_annotations_path) as f:
    coco_annotations = json.load(f)

def filter_coco_annotations(files, annotations):
    filtered = {'images': [], 'annotations': [], 'categories': annotations['categories']}
    image_ids = {os.path.splitext(f)[0] for f in files}
    filtered['images'] = [image for image in annotations['images'] if image['file_name'].replace('.jpg', '') in image_ids]
    filtered_image_ids = {image['id'] for image in filtered['images']}
    filtered['annotations'] = [ann for ann in annotations['annotations'] if ann['image_id'] in filtered_image_ids]
    return filtered

def organize_dataset(files, source_images_dir, yolo_dir, images_dest_dir, labels_dest_dir):
    for filename in files:
        # Copy image
        shutil.copy(os.path.join(source_images_dir, filename), os.path.join(images_dest_dir, filename))
        
        # Copy YOLO annotation
        yolo_annotation_filename = filename.replace('.jpg', '.txt')
        shutil.copy(os.path.join(yolo_dir, yolo_annotation_filename), os.path.join(labels_dest_dir, yolo_annotation_filename))

# Filter COCO annotations for train and val sets
train_annotations_coco = filter_coco_annotations(train_files, coco_annotations)
val_annotations_coco = filter_coco_annotations(val_files, coco_annotations)

# Organize train and validation datasets
organize_dataset(train_files, source_images_dir, yolo_annotations_dir, os.path.join(images_dir, 'train'), os.path.join(labels_dir, 'train'))
organize_dataset(val_files, source_images_dir, yolo_annotations_dir, os.path.join(images_dir, 'val'), os.path.join(labels_dir, 'val'))

# Save filtered COCO annotations
with open(os.path.join(coco_dir, 'train.json'), 'w') as f:
    json.dump(train_annotations_coco, f)
with open(os.path.join(coco_dir, 'val.json'), 'w') as f:
    json.dump(val_annotations_coco, f)

print("Dataset organized with separate directories for images, YOLO labels, and COCO annotations.")


Dataset organized with separate directories for images, YOLO labels, and COCO annotations.


In [13]:
# The path to the original YAML file
input_yaml_path = '/kaggle/input/customdataset/ObjectDetection/google_colab_files.yaml'
# The path where the modified YAML file will be saved
output_yaml_path = '/kaggle/working/google_colab_files.yaml'

# Load the original YAML file
with open(input_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

# Modify the dataset configuration as needed
data['path'] = '/kaggle/working'
data['val'] = 'images/val'  # Assuming you want to point to the validation dataset

# Save the modified configuration to a new YAML file
with open(output_yaml_path, 'w') as file:
    yaml.dump(data, file, sort_keys=False)  # sort_keys=False to preserve the order of keys


## YOLO configuration & Training

In [14]:
# Load a model configuration and a pre-trained model
model = YOLO("yolov8n.yaml")  # Adjust this to your specific configuration file if necessary
model = YOLO("yolov8n.pt")  # Load a pre-trained model

In [15]:
model.train(data='/kaggle/working/google_colab_files.yaml', epochs=100)

# Optionally, perform validation
model.val()

Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/google_colab_files.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

wandb: Currently logged in as: samyukta-palle (aipi549). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/labels/train... 92 images, 1 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<00:00, 168.04it/s]

train: WARNING ⚠️ /kaggle/working/images/train/20210923_134958.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20220408_145156.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20221023_192449.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240208_120437.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240226_143418.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20211111-WA0005.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0020.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0025.jpg: corrupt JPEG restored and saved
train: New cache created: /kaggle/working/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limi

...

val: Scanning /kaggle/working/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/images/val/IMG-20220409-WA0005.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.64s/it]


                   all         24         35      0.775      0.745      0.763      0.604
                 drink         24          9      0.783      0.667        0.7       0.58
               utensil         24         11      0.731      0.636      0.688      0.429
                laptop         24         15       0.81      0.933      0.899      0.804
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train32


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c46b5fae470>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### Saving the runs folder

In [16]:
!zip -r runs.zip /kaggle/working/runs/detect

updating: kaggle/working/runs/detect/ (stored 0%)
updating: kaggle/working/runs/detect/train/ (stored 0%)
updating: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 3%)
updating: kaggle/working/runs/detect/train/results.png (deflated 6%)
updating: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 48%)
updating: kaggle/working/runs/detect/train/train_batch542.jpg (deflated 10%)
updating: kaggle/working/runs/detect/train/train_batch541.jpg (deflated 8%)
updating: kaggle/working/runs/detect/train/PR_curve.png (deflated 19%)
updating: kaggle/working/runs/detect/train/events.out.tfevents.1711336623.2bc2fb90ec96.34.0 (deflated 87%)
updating: kaggle/working/runs/detect/train/F1_curve.png (deflated 10%)
updating: kaggle/working/runs/detect/train/val_batch0_labels.jpg (deflated 10%)
updating: kaggle/working/runs/detect/train/train_batch540.jpg (deflated 7%)
updating: kaggle/working/runs/detect/train/P_curve.png (deflated 11%)
updating: kaggle/working/runs/detect/train

In [17]:
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr

def __init__(self, p=1.0):
        """Initialize the transform object for YOLO bbox formatted params."""
        self.p = p
        self.transform = None
        prefix = colorstr("albumentations: ")
        try:
            import albumentations as A

            # check_version(A.__version__, "1.0.3", hard=True)  # version requirement

            # Transforms
            T = [
                A.Rotate(limit = 10, p=0.5),
                A.Blur(p=0.2),
                A.MedianBlur(p=0.2),
                A.ToGray(p=0.2),
                A.CLAHE(p=0.2),
                A.RandomBrightnessContrast(p=0.2),
                A.RandomGamma(p=0.2),
                A.ImageCompression(quality_lower=75, p=0.0),
            ]
            self.transform = A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

            LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
        except ImportError:  # package not installed, skip
            pass
        except Exception as e:
            LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = __init__

model = YOLO('yolov8n.yaml').load('yolov8n.pt')

model.train(data='/kaggle/working/google_colab_files.yaml', epochs=100)

# Optionally, perform validation
model.val()


Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/google_colab_files.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/labels/train.cache... 92 images, 1 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/working/images/train/20210923_134958.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20220408_145156.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20221023_192449.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240208_120437.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240226_143418.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20211111-WA0005.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0020.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0025.jpg: corrupt JPEG restored and saved
albumentations: Rotate(p=0.5, limit=(-10, 10), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box', crop_border=False), Blur(p=0.2, blur_limit=(3, 7)),


val: Scanning /kaggle/working/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

...

val: Scanning /kaggle/working/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/images/val/IMG-20220409-WA0005.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:05<00:00,  2.54s/it]


                   all         24         35      0.855      0.678      0.757      0.532
                 drink         24          9      0.735      0.556      0.641      0.547
               utensil         24         11      0.951      0.545      0.697      0.285
                laptop         24         15      0.879      0.933      0.935      0.766
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/detect/train42


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c455d539990>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [19]:
from ultralytics.data.augment import Albumentations
from ultralytics.utils import LOGGER, colorstr

def __init__(self, p=1.0):
        """Initialize the transform object for YOLO bbox formatted params."""
        self.p = p
        self.transform = None
        prefix = colorstr("albumentations: ")
        try:
            import albumentations as A

            # check_version(A.__version__, "1.0.3", hard=True)  # version requirement

            # Transforms
            T = [
                A.Rotate(limit = 10, p=0.2),
                A.Blur(p=0.2),
                A.MedianBlur(p=0.2),
                A.ToGray(p=0.2),
                A.CLAHE(p=0.2),
                A.RandomBrightnessContrast(p=0.2),
                A.RandomGamma(p=0.2),
                A.ImageCompression(quality_lower=75, p=0.0),
            ]
            self.transform = A.Compose(T, bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]))

            LOGGER.info(prefix + ", ".join(f"{x}".replace("always_apply=False, ", "") for x in T if x.p))
        except ImportError:  # package not installed, skip
            pass
        except Exception as e:
            LOGGER.info(f"{prefix}{e}")

Albumentations.__init__ = __init__

model = YOLO('yolov8n.yaml').load('yolov8n.pt')

model.train(data='/kaggle/working/google_colab_files.yaml', epochs=100)

# Optionally, perform validation
model.val()


Transferred 355/355 items from pretrained weights
Ultralytics YOLOv8.1.34 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/kaggle/working/google_colab_files.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, 

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅


train: Scanning /kaggle/working/labels/train.cache... 92 images, 1 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/working/images/train/20210923_134958.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20220408_145156.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20221023_192449.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240208_120437.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/20240226_143418.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20211111-WA0005.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0020.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /kaggle/working/images/train/IMG-20240202-WA0025.jpg: corrupt JPEG restored and saved
albumentations: Rotate(p=0.2, limit=(-10, 10), interpolation=1, border_mode=4, value=None, mask_value=None, rotate_method='largest_box', crop_border=False), Blur(p=0.2, blur_limit=(3, 7)),


val: Scanning /kaggle/working/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

...

val: Scanning /kaggle/working/labels/val.cache... 24 images, 0 backgrounds, 0 corrupt: 100%|██████████| 24/24 [00:00<?, ?it/s]

val: WARNING ⚠️ /kaggle/working/images/val/IMG-20220409-WA0005.jpg: corrupt JPEG restored and saved



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:04<00:00,  2.43s/it]


                   all         24         35      0.696      0.774      0.752      0.573
                 drink         24          9      0.746      0.654      0.686       0.52
               utensil         24         11      0.502      0.734      0.633      0.297
                laptop         24         15      0.839      0.933      0.935      0.902
Speed: 0.2ms preprocess, 2.6ms inference, 0.0ms loss, 3.9ms postprocess per image
Results saved to runs/detect/train62


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c459e2b7070>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [20]:
!zip -r runs.zip /kaggle/working/runs/detect

updating: kaggle/working/runs/detect/ (stored 0%)
updating: kaggle/working/runs/detect/train/ (stored 0%)
updating: kaggle/working/runs/detect/train/train_batch0.jpg (deflated 3%)
updating: kaggle/working/runs/detect/train/results.png (deflated 6%)
updating: kaggle/working/runs/detect/train/labels_correlogram.jpg (deflated 48%)
updating: kaggle/working/runs/detect/train/train_batch542.jpg (deflated 10%)
updating: kaggle/working/runs/detect/train/train_batch541.jpg (deflated 8%)
updating: kaggle/working/runs/detect/train/PR_curve.png (deflated 19%)
updating: kaggle/working/runs/detect/train/events.out.tfevents.1711336623.2bc2fb90ec96.34.0 (deflated 87%)
updating: kaggle/working/runs/detect/train/F1_curve.png (deflated 10%)
updating: kaggle/working/runs/detect/train/val_batch0_labels.jpg (deflated 10%)
updating: kaggle/working/runs/detect/train/train_batch540.jpg (deflated 7%)
updating: kaggle/working/runs/detect/train/P_curve.png (deflated 11%)
updating: kaggle/working/runs/detect/train